In [76]:
import pandas as pd
import numpy as np
from scipy.stats import t, pearsonr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import chisquare, poisson 

In [178]:
df = pd.read_csv('./Datasets/Hurricane.csv')
df.head()

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,"August, September",150,924
1,"""1856 Last Island Hurricane""",1856,August,150,934
2,Hurricane #6,1866,"September, October",140,938
3,Hurricane #7,1878,"September, October",140,938
4,Hurricane #2,1880,August,150,931


In [158]:
df.shape

(101, 5)

In [179]:
df = df.assign(Month=df['Month'].str.split(', ')).explode('Month').reset_index(drop=True)
df.head()

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,August,150,924
1,Hurricane #3,1853,September,150,924
2,"""1856 Last Island Hurricane""",1856,August,150,934
3,Hurricane #6,1866,September,140,938
4,Hurricane #6,1866,October,140,938


In [180]:
df_normalize = df.copy()

In [160]:
df.shape

(137, 5)

In [161]:
df_normalize.shape

(137, 5)

In [126]:
from sklearn.preprocessing import LabelEncoder

In [127]:
# # Initialize LabelEncoder
# label_encoder = LabelEncoder()


# # Apply LabelEncoder to the 'Month' column
# df_normalize['Month'] = label_encoder.fit_transform(df_normalize['Month'])
# df['Month'] = label_encoder.fit_transform(df['Month'])

In [181]:
month_mapping = {
    'July' : 0,
    'August': 1,
    'September': 2,
    'October': 3,
    'November': 4,
    'December': 5
}

# Create a new column with the integer mapping
df['Month'] = df['Month'].map(month_mapping)

In [155]:
df['Month'].value_counts()

Series([], Name: Month, dtype: int64)

In [163]:
df.head()

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,1,150,924
1,Hurricane #3,1853,2,150,924
2,"""1856 Last Island Hurricane""",1856,1,150,934
3,Hurricane #6,1866,2,140,938
4,Hurricane #6,1866,3,140,938


In [138]:
df_normalize.head()

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,-2.471014,0,0.990500,-1.487932
1,Hurricane #3,-2.471014,4,0.990500,-1.487932
2,"""1856 Last Island Hurricane""",-2.395179,0,0.990500,-0.482771
3,Hurricane #6,-2.142394,4,-0.301867,-0.080706
4,Hurricane #6,-2.142394,3,-0.301867,-0.080706


In [164]:
def mean_of_data(column):
    total = 0
    for i in column:
        total += i
    return total/len(column)

def variance_of_data(column):
    mean = mean_of_data(column)
    total = 0
    for i in column:
        total += (i - mean)**2
    return total/len(column)

In [165]:
# def normalize_feature():
#     for column in df_normalize.columns[1:]:
#         mean = mean_of_data(df_normalize[column])
#         variance = variance_of_data(df_normalize[column])
#         standard_dev = variance**0.5
#         df_normalize[column] = (df_normalize[column] - mean) / standard_dev

In [183]:
def normalize_feature():
    for column in df_normalize.columns[1:]:
        if column != 'Month':  # Skip the first column if it's an index or non-feature
            min_value = df_normalize[column].min()
            max_value = df_normalize[column].max()
        # Apply min-max scaling
            df_normalize[column] = (df_normalize[column] - min_value) / (max_value - min_value)


In [184]:
normalize_feature()

In [185]:
df

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,1,150,924
1,Hurricane #3,1853,2,150,924
2,"""1856 Last Island Hurricane""",1856,1,150,934
3,Hurricane #6,1866,2,140,938
4,Hurricane #6,1866,3,140,938
...,...,...,...,...,...
132,Hurricane Fabian,2003,2,145,939
133,Hurricane Charley,2004,1,150,941
134,Hurricane Frances,2004,1,145,935
135,Hurricane Frances,2004,2,145,935


In [186]:
df_normalize

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,0.000000,August,0.8,0.135593
1,Hurricane #3,0.000000,September,0.8,0.135593
2,"""1856 Last Island Hurricane""",0.019868,August,0.8,0.305085
3,Hurricane #6,0.086093,September,0.4,0.372881
4,Hurricane #6,0.086093,October,0.4,0.372881
...,...,...,...,...,...
132,Hurricane Fabian,0.993377,September,0.6,0.389831
133,Hurricane Charley,1.000000,August,0.8,0.423729
134,Hurricane Frances,1.000000,August,0.6,0.322034
135,Hurricane Frances,1.000000,September,0.6,0.322034


In [187]:
month_mapping = {
    'July' : 0,
    'August': 1,
    'September': 2,
    'October': 3,
    'November': 4,
    'December': 5
}

# Create a new column with the integer mapping
df_normalize['Month'] = df_normalize['Month'].map(month_mapping)

a. With a 1% level of significance conduct t-test for correlation coefficient between “Max. sustained winds(mph)” and “Minimum pressure (mbar)”.

In [144]:
df_normalize

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,-2.471014,-1.623394,0.990500,-1.487932
1,Hurricane #3,-2.471014,0.787500,0.990500,-1.487932
2,"""1856 Last Island Hurricane""",-2.395179,-1.623394,0.990500,-0.482771
3,Hurricane #6,-2.142394,0.787500,-0.301867,-0.080706
4,Hurricane #6,-2.142394,0.184777,-0.301867,-0.080706
...,...,...,...,...,...
132,Hurricane Fabian,1.320753,0.787500,0.344317,0.019810
133,Hurricane Charley,1.346031,-1.623394,0.990500,0.220842
134,Hurricane Frances,1.346031,-1.623394,0.344317,-0.382255
135,Hurricane Frances,1.346031,0.787500,0.344317,-0.382255


In [92]:
alpha = 0.01
n = df_normalize.shape[0]
r = np.corrcoef(df_normalize["Max. sustained winds(mph)"], df_normalize["Minimum pressure(mbar)"])[0, 1]
t_value = r / np.sqrt((1 - r**2) / (n - 2))
t_value

-6.067728544364718

In [93]:
# Calculate degrees of freedom
degrees = n - 2

# Two-tailed p-value
p_value = 2 * t.cdf(-np.abs(t_value), degrees)
p_value

1.230390463474083e-08

In [144]:
p_value < alpha

True

In [7]:
alpha = 0.01
n = df.shape[0]
r = np.corrcoef(df["Max. sustained winds(mph)"], df["Minimum pressure(mbar)"])[0, 1]
t_value = r / np.sqrt((1 - r**2) / (n - 2))
t_value

-6.067728544364715

In [94]:
r

-0.4629058408860271

In [95]:
degrees = n-2

critical_value = t.ppf(1 - alpha/2, degrees)

# Print the critical value
print(f"The critical value is: {critical_value:.3f}")

The critical value is: 2.613


In [96]:
if (t_value < critical_value):
    print("Rejecting the NULL Hypothesis")
else:
    print("Failed to reject the NULL Hypothesis")

Rejecting the NULL Hypothesis


In [146]:
# Calculate degrees of freedom
degrees = n - 2

# Two-tailed p-value
p_value2 = 2 * t.cdf(-np.abs(t_value), degrees)
p_value2

1.2303904634741104e-08

In [147]:
test_result = pearsonr(df["Max. sustained winds(mph)"], df["Minimum pressure(mbar)"])
test_result

PearsonRResult(statistic=-0.4629058408860275, pvalue=1.2303904634740368e-08)

In [148]:
test_result.pvalue

1.2303904634740368e-08

In [149]:
test_result.confidence_interval()

ConfidenceInterval(low=-0.5851892925012572, high=-0.3200368904005092)

In [150]:
# Extract relevant columns
winds = df['Max. sustained winds(mph)']
pressure = df['Minimum pressure(mbar)']

# Calculate Pearson correlation
correlation, p_value = pearsonr(winds, pressure)

# Display the result
print(f"Pearson correlation: {correlation}")
print(f"P-value: {p_value}")

# Test the significance at 1% level
if p_value < 0.01:
    print("Reject the null hypothesis: Significant correlation.")
else:
    print("Fail to reject the null hypothesis: No significant correlation.")


Pearson correlation: -0.4629058408860275
P-value: 1.2303904634740368e-08
Reject the null hypothesis: Significant correlation.


In [151]:
df.drop(columns=['Name', 'Month']).corr()['Max. sustained winds(mph)'].sort_values(ascending=False)

Max. sustained winds(mph)    1.000000
Season                       0.066874
Minimum pressure(mbar)      -0.462906
Name: Max. sustained winds(mph), dtype: float64

b. With a 5% level of significance test if the “Max. sustained winds(mph)” of hurricane depends on the month of its occurrence.

In [97]:
df

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,0,150,924
1,Hurricane #3,1853,4,150,924
2,"""1856 Last Island Hurricane""",1856,0,150,934
3,Hurricane #6,1866,4,140,938
4,Hurricane #6,1866,3,140,938
...,...,...,...,...,...
132,Hurricane Fabian,2003,4,145,939
133,Hurricane Charley,2004,0,150,941
134,Hurricane Frances,2004,0,145,935
135,Hurricane Frances,2004,4,145,935


In [98]:
df

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,0,150,924
1,Hurricane #3,1853,4,150,924
2,"""1856 Last Island Hurricane""",1856,0,150,934
3,Hurricane #6,1866,4,140,938
4,Hurricane #6,1866,3,140,938
...,...,...,...,...,...
132,Hurricane Fabian,2003,4,145,939
133,Hurricane Charley,2004,0,150,941
134,Hurricane Frances,2004,0,145,935
135,Hurricane Frances,2004,4,145,935


In [145]:
df

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,0,150,924
1,Hurricane #3,1853,4,150,924
2,"""1856 Last Island Hurricane""",1856,0,150,934
3,Hurricane #6,1866,4,140,938
4,Hurricane #6,1866,3,140,938
...,...,...,...,...,...
132,Hurricane Fabian,2003,4,145,939
133,Hurricane Charley,2004,0,150,941
134,Hurricane Frances,2004,0,145,935
135,Hurricane Frances,2004,4,145,935


In [152]:
df['Month'].value_counts

<bound method IndexOpsMixin.value_counts of 0      0
1      4
2      0
3      4
4      3
      ..
132    4
133    0
134    0
135    4
136    4
Name: Month, Length: 137, dtype: int32>

In [176]:
df_normalize

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,0.000000,0.00,0.8,0.135593
1,Hurricane #3,0.000000,1.00,0.8,0.135593
2,"""1856 Last Island Hurricane""",0.019868,0.00,0.8,0.305085
3,Hurricane #6,0.086093,1.00,0.4,0.372881
4,Hurricane #6,0.086093,0.75,0.4,0.372881
...,...,...,...,...,...
132,Hurricane Fabian,0.993377,1.00,0.6,0.389831
133,Hurricane Charley,1.000000,0.00,0.8,0.423729
134,Hurricane Frances,1.000000,0.00,0.6,0.322034
135,Hurricane Frances,1.000000,1.00,0.6,0.322034


In [188]:
df_normalize

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,0.000000,1,0.8,0.135593
1,Hurricane #3,0.000000,2,0.8,0.135593
2,"""1856 Last Island Hurricane""",0.019868,1,0.8,0.305085
3,Hurricane #6,0.086093,2,0.4,0.372881
4,Hurricane #6,0.086093,3,0.4,0.372881
...,...,...,...,...,...
132,Hurricane Fabian,0.993377,2,0.6,0.389831
133,Hurricane Charley,1.000000,1,0.8,0.423729
134,Hurricane Frances,1.000000,1,0.6,0.322034
135,Hurricane Frances,1.000000,2,0.6,0.322034


In [189]:
# Calculate correlation coefficient r
r = np.corrcoef(df_normalize['Max. sustained winds(mph)'],df_normalize['Month'] )[0, 1]
alpha = 0.05
# Calculate the t-value
n = df_normalize.shape[0]
t_value = r * np.sqrt((n - 2) / (1 - r**2))
degrees = n-2
# Calculate the critical value
critical_value = t.ppf(1 - alpha/2, degrees)

# Output results
print(f"Correlation coefficient (r): {r}")
print(f"Calculated t-value: {t_value}")
print(f"Critical t-value at 5% significance level: {critical_value}")

# Decision
if  abs(t_value) > critical_value:
    print("Reject the null hypothesis: Max sustained winds depend on the month of occurrence.")
else:
    print("Fail to reject the null hypothesis: No significant dependence on the month.")

Correlation coefficient (r): 0.009000113479535417
Calculated t-value: 0.10457610438545947
Critical t-value at 5% significance level: 1.977692277222804
Fail to reject the null hypothesis: No significant dependence on the month.


In [191]:
df

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,1,150,924
1,Hurricane #3,1853,2,150,924
2,"""1856 Last Island Hurricane""",1856,1,150,934
3,Hurricane #6,1866,2,140,938
4,Hurricane #6,1866,3,140,938
...,...,...,...,...,...
132,Hurricane Fabian,2003,2,145,939
133,Hurricane Charley,2004,1,150,941
134,Hurricane Frances,2004,1,145,935
135,Hurricane Frances,2004,2,145,935


In [190]:
# Calculate correlation coefficient r
r = np.corrcoef(df['Max. sustained winds(mph)'],df['Month'] )[0, 1]
alpha = 0.05
# Calculate the t-value
n = df_normalize.shape[0]
t_value = r * np.sqrt((n - 2) / (1 - r**2))
degrees = n-2
# Calculate the critical value
critical_value = t.ppf(1 - alpha/2, degrees)

# Output results
print(f"Correlation coefficient (r): {r}")
print(f"Calculated t-value: {t_value}")
print(f"Critical t-value at 5% significance level: {critical_value}")

# Decision
if  abs(t_value) > critical_value:
    print("Reject the null hypothesis: Max sustained winds depend on the month of occurrence.")
else:
    print("Fail to reject the null hypothesis: No significant dependence on the month.")

Correlation coefficient (r): 0.009000113479535408
Calculated t-value: 0.10457610438545938
Critical t-value at 5% significance level: 1.977692277222804
Fail to reject the null hypothesis: No significant dependence on the month.


In [34]:
winds = df_normalize['Max. sustained winds(mph)']

month = df_normalize['Month']

alpha = 0.05
n = df.shape[0]
r = np.corrcoef(winds, month)[0, 1]
t_value = r / np.sqrt((1 - r**2) / (n - 2))
t_value


-0.7407173211276721

In [36]:
r

-0.06362164333733293

In [30]:
degrees = n-2

critical_value = t.ppf(1 - alpha/2, degrees)

# Print the critical value
print(f"The critical value is: {critical_value:.3f}")

The critical value is: 1.978


In [31]:
if (t_value < critical_value):
    print("Rejecting the NULL Hypothesis")
else:
    print("Failed to reject the NULL Hypothesis")

Rejecting the NULL Hypothesis


In [152]:
# Extract relevant columns
winds = df_normalize['Max. sustained winds(mph)']
month = df_normalize['Month']

# Calculate Pearson correlation
correlation, p_value = pearsonr(winds, month)

# Display the result
print(f"Pearson correlation: {correlation}")
print(f"P-value: {p_value}")

# Test the significance at 1% level
if p_value < 0.05:
    print("Reject the null hypothesis: Significant correlation.")
else:
    print("Fail to reject the null hypothesis: No significant correlation.")


Pearson correlation: -0.06362164333733303
P-value: 0.4601512540636204
Fail to reject the null hypothesis: No significant correlation.


In [153]:
model = ols('Q("Max. sustained winds(mph)") ~ C(Month)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

In [154]:
anova_table

,sum_sq,df,F,PR(>F)
C(Month),94.000823,4.0,0.382562,0.820795
Residual,8108.553922,132.0,NaN,NaN


In [155]:
p_value = anova_table['PR(>F)'][0]
print(p_value)
if p_value < 0.05:
    print("Reject the null hypothesis: Winds depend on the month.")
else:
    print("Fail to reject the null hypothesis: Winds do not depend on the month.")

0.8207947187971213
Fail to reject the null hypothesis: Winds do not depend on the month.


c. With a 10% level of significance conduct test if “Max. sustained winds(mph)” follows a Poisson distribution.

In [156]:
df.shape

(137, 5)

In [157]:
df.head()

,Name,Season,Month,Max. sustained winds(mph),Minimum pressure(mbar)
0,Hurricane #3,1853,August,150,924
1,Hurricane #3,1853,September,150,924
2,"""1856 Last Island Hurricane""",1856,August,150,934
3,Hurricane #6,1866,September,140,938
4,Hurricane #6,1866,October,140,938


In [202]:
import pandas as pd
from scipy.stats import poisson, chisquare

# Assuming df is already defined and contains 'Max. sustained winds(mph)'
winds = df['Max. sustained winds(mph)']

mean_wind = winds.mean()
print(mean_wind)
# Generate observed frequencies (assuming wind speeds are integers)
observed_freq = winds.value_counts().sort_index()
print("Observed Frequencies:\n", observed_freq)

# Generate Poisson probabilities for the observed values
total = len(winds)
expected_freq = [poisson.pmf(k, mean_wind)  for k in observed_freq.index]
total2 = sum(expected_freq)
expected_freq = [prob * total/total2 for prob in expected_freq]
print("Expected Frequencies:\n", expected_freq)

# Check sums
print("Sum of Observed Frequencies:", observed_freq.sum())
print("Sum of Expected Frequencies:", sum(expected_freq))

# Ensure lengths are the same
if len(observed_freq) != len(expected_freq):
    print("Length mismatch: Adjust expected frequencies to match observed frequencies.")
else:
    chi_square_stat, p_value = chisquare(f_obs=observed_freq, f_exp=expected_freq)

    print(f"Chi-square statistic: {chi_square_stat}")
    print(f"P-value: {p_value}")

    # Check significance at 10% level
    if p_value < 0.10:
        print("Reject the null hypothesis: Winds do not follow a Poisson distribution.")
    else:
        print("Fail to reject the null hypothesis: Winds follow a Poisson distribution.")


142.33576642335765
Observed Frequencies:
 130    28
140    39
145    33
150    24
155    13
Name: Max. sustained winds(mph), dtype: int64
Expected Frequencies:
 [21.14459376472458, 34.66753441054795, 33.87811106758004, 27.879178668675344, 19.430582088472068]
Sum of Observed Frequencies: 137
Sum of Expected Frequencies: 137.0
Chi-square statistic: 5.4547959472372565
P-value: 0.2437316198723859
Fail to reject the null hypothesis: Winds follow a Poisson distribution.


In [204]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

# Assuming df is already defined and contains 'Max. sustained winds(mph)'
winds = df['Max. sustained winds(mph)']

mean_wind = winds.mean()

# Generate observed frequencies (assuming wind speeds are integers)
observed_freq = winds.value_counts().sort_index()
print("Observed Frequencies:\n", observed_freq)

# Generate Poisson probabilities for the observed values
total = len(winds)
expected_freq = [poisson.pmf(k, mean_wind) * total for k in observed_freq.index]

# Ensure expected_freq is a Series with the same index as observed_freq
expected_freq_series = pd.Series(expected_freq, index=observed_freq.index).fillna(0)

print("Expected Frequencies:\n", expected_freq_series)

# Check sums
print("Sum of Observed Frequencies:", observed_freq.sum())
print("Sum of Expected Frequencies:", expected_freq_series.sum())

# Calculate chi-square statistic from scratch
chi_square_stat = np.sum((observed_freq - expected_freq_series) ** 2 / expected_freq_series)

print(f"Chi-square statistic (manual calculation): {chi_square_stat}")

# Degrees of freedom
degrees_of_freedom = len(observed_freq) - 1
alpha = 0.10
critical_value = chi2.ppf(1 - alpha, degrees_of_freedom)
print(f"Critical value at {alpha*100}% significance level: {critical_value}")

# Calculate p-value using the chi-square distribution
from scipy.stats import chi2
p_value = 1 - chi2.cdf(chi_square_stat, degrees_of_freedom)

print(f"P-value (manual calculation): {p_value}")

# Check significance at 10% level
if p_value < 0.10:
    print("Reject the null hypothesis: Winds do not follow a Poisson distribution.")
else:
    print("Fail to reject the null hypothesis: Winds follow a Poisson distribution.")


Observed Frequencies:
 130    28
140    39
145    33
150    24
155    13
Name: Max. sustained winds(mph), dtype: int64
Expected Frequencies:
 130    2.761946
140    4.528338
145    4.425222
150    3.641630
155    2.538059
dtype: float64
Sum of Observed Frequencies: 137
Sum of Expected Frequencies: 17.895195365650657
Chi-square statistic (manual calculation): 834.4844091739852
Critical value at 10.0% significance level: 7.779440339734858
P-value (manual calculation): 0.0
Reject the null hypothesis: Winds do not follow a Poisson distribution.


In [50]:
import numpy as np
from scipy import stats

In [131]:
df['Max. sustained winds(mph)'].describe()

count    137.000000
mean     142.335766
std        7.766138
min      130.000000
25%      140.000000
50%      145.000000
75%      150.000000
max      155.000000
Name: Max. sustained winds(mph), dtype: float64

In [133]:
from scipy.special import gammaln 

In [136]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import gammaln
data = df['Max. sustained winds(mph)']
# Assuming 'data' and 'n' are defined elsewhere in your code
mean_lambda = np.mean(data)

# Step 3: Create the Frequency Table
values, frequencies = np.unique(data, return_counts=True)
frequency_table = pd.DataFrame({'Wind_Speed': values, 'Observed_Frequency': frequencies})

# Step 4: Calculate Expected Frequencies
expected_frequencies = []
for k in values:
    expected_freq = (np.exp(-mean_lambda) * (mean_lambda**k) / np.exp(gammaln(k + 1))) * n
    expected_frequencies.append(expected_freq)

frequency_table['Expected_Frequency'] = expected_frequencies

# Step 5: Perform the Chi-Squared Goodness-of-Fit Test
chi_squared_stat = sum((frequency_table['Observed_Frequency'] - frequency_table['Expected_Frequency']) ** 2 / frequency_table['Expected_Frequency'])
degrees = len(frequency_table) - 1 - 1  # K - 1 - p, p = 1 for lambda
alpha = 0.10

# Step 6: Find the Critical Value
critical_value = stats.chi2.ppf(1 - alpha, degrees)

# Step 7: Decision Rule
if chi_squared_stat > critical_value:
    print("Reject the null hypothesis: The data does not follow a Poisson distribution.")
else:
    print("Do not reject the null hypothesis: The data may follow a Poisson distribution.")

# Output the results
print(f"Chi-Squared Statistic: {chi_squared_stat:.4f}")
print(f"Critical Value at 10% significance level: {critical_value:.4f}")

# Optional: Display the frequency table
print(frequency_table)

Do not reject the null hypothesis: The data may follow a Poisson distribution.
Chi-Squared Statistic: nan
Critical Value at 10% significance level: 6.2514
   Wind_Speed  Observed_Frequency  Expected_Frequency
0         130                  28            2.761946
1         140                  39            4.528338
2         145                  33                 inf
3         150                  24                 inf
4         155                  13                 inf


C:\Users\aarzo\AppData\Local\Temp\ipykernel_29472\1126396723.py:16: RuntimeWarning: overflow encountered in scalar power
  expected_freq = (np.exp(-mean_lambda) * (mean_lambda**k) / np.exp(gammaln(k + 1))) * n


In [116]:
import numpy as np
from scipy import stats

def chi_square_poisson_test(data, significance_level=0.10):
    observed_freq, bins = np.histogram(data, bins='auto')
    expected_freq = np.diff(stats.poisson.cdf(bins, np.mean(data))) * len(data)
    
    combined_obs = {}
    combined_exp = {}
    current_obs = 0
    current_exp = 0
    
    for x in range(len(observed_freq)):
        current_obs += observed_freq[x]
        current_exp += expected_freq[x]
        if current_exp >= 5:
            combined_obs[x] = current_obs
            combined_exp[x] = current_exp
            current_obs = 0
            current_exp = 0
    
    # Calculate chi-square statistic
    chi_square = sum((combined_obs[x] - combined_exp[x])**2 / combined_exp[x] for x in combined_obs)
    
    # Calculate degrees of freedom
    degrees = len(combined_obs) - 1 - 1  # Subtract 1 for estimated parameter (mean)
    
    # Calculate p-value
    p_value = 1 - stats.chi2.cdf(chi_square, degrees)
    
    # Get critical value
    critical_value = stats.chi2.ppf(1 - significance_level, degrees)
    
    return chi_square, degrees, p_value, critical_value

# Example usage:
# Ensure the DataFrame is loaded correctl

# Example: Load the DataFrame (replace with your actual data loading code)
# df = pd.read_csv('your_data_file.csv')

# Check if the column exists
if 'Max. sustained winds(mph)' in df.columns:

    data = df['Max. sustained winds(mph)']
    
    chi_square, degrees, p_value, critical_value = chi_square_poisson_test(data)

    print(f"Chi-square statistic: {chi_square:.4f}")
    print(f"Degrees of freedom: {degrees}")
    print(f"P-value: {p_value:.4f}")
    print(f"Critical value at 10% significance: {critical_value:.4f}")

    if chi_square > critical_value:
        print("Reject the null hypothesis. The data does not follow a Poisson distribution.")
    else:
        print("Fail to reject the null hypothesis. The data may follow a Poisson distribution.")
else:
    print("Column 'Max. sustained winds(mph)' does not exist in the DataFrame.")

Chi-square statistic: 218.9336
Degrees of freedom: 7
P-value: 0.0000
Critical value at 10% significance: 12.0170
Reject the null hypothesis. The data does not follow a Poisson distribution.
